
# Atmospheric grid



In this section, we indicate the applicability range of the default atmospheric grid, as well as the possible error messages associated to the atmospheric grid and what they mean. Finally, if the default grid does not cover the parameter space that you need (i.e higher equilibrium temperatures, higher metallicity in the envelope) or you want to change the assumptions on the atmosphere (i.e include clouds), we explain how you can use your custom atmospheric grid with GASTLI.


Default grid
============

Applicability
-------------

The range of parameter values convered by the default grid of atmospheric models is indicated in the table below. The limiting log-metallicities correspond to an almost pure H/He envelope (0.01 x solar) to a 80% mass fraction in metals (250 x solar). The surface gravity of an interior-atmosphere model is not known a priori since it depends on the interior composition that is assumed. However, we can provide approximate estimates: for a log-surface gravity of 2.6 cgs, the mass is approximately 15-20 Earth masses, while a log-surface gravity of 4.18 cgs corresponds to planets between 5 and 6.5 Jupiter masses.  

.. figure:: petitcode_grid.jpeg
   :align: center

   Table 1 in Acuña et al. Submitted, which shows the applicability of GASTLI's default atmospheric grid.

The atmospheric grid is interpolated using Python's ``scipy.interpolate.RegularGridInterpolator``, with the flags ``bounds_error=False`` and ``fill_value=None``. This means that if a parameter is outside of the atmospheric grid domain, extrapolation is performed. In that case, GASTLI will not stop the calculation, but it will display a message such as this: 

.. code-block:: language

   Surface gravity is out of atmospheric grid limits. Extrapolating


Troubleshooting
---------------

The default surface pressure in the atmospheric grid and the interior-atmosphere coupling is 1000 bar. However, such high pressure can be challenging for the convergence of some atmospheric models, especially at low surface gravities, high log-metallicities, and high internal temperatures. For these models, we fill the pressures between their maximum possible pressure (9.5 bar) and 1000 bar with ``np.nan``. This enables us to keep the atmospheric grid regular in 6 dimensions. Regular grids are faster to interpolate than irregular grids. If your interior-atmosphere model requires interpolating atmospheric models for which 9.5 bar is the maximum pressure, but you have the default surface pressure of 1000 bar, GASTLI will stop the computation and show the following message:

.. code-block:: language

   No atmospheric models available for this case (np.nan in grid).
   Decrease the interior temperature or decrease the surface pressure

If you try to run this same model without specifying ``P_surf``, the default of 1000 bar will be assumed, and the error above will be shown. Hence, we recommend to decrease the surface pressure by specifying it in the ``main()`` function of the coupling class. Example:


In [1]:
# Import coupling module
import gastli.Coupling as cpl
# Other Python modules
import numpy as np
# Create coupling class
my_coupling = cpl.coupling(j_max=99, pow_law_formass=0.31)
# Input for interior
M_P = 50.
# Internal and equilibrium temperatures
Tintpl = 700.
Teqpl = 1000.
# Core mass fraction
CMF = 0.5
# Run model with P_surf at 9.5 bar
my_coupling.main(M_P, CMF, Teqpl, Tintpl, CO=0.55, log_FeH=2.4,Rguess=6.,P_surf=9.5)


Running interior structure model
 [i] Allowed maximum number of iterations reached.
 

Running interior structure model
 [i] Allowed maximum number of iterations reached.
 

Running interior structure model
 [i] Allowed maximum number of iterations reached.
 

Running interior structure model
 [i] Allowed maximum number of iterations reached.
 

Convergence reached in surface temperature and bulk radius



If you are running a thermal evolution class, you can specify at which surface pressures you want each of the model of the sequence to be calculated at. This is done by setting ``P_surf`` to an array of the same length as ``Tint_array`` in the ``main()`` function of the thermal evolution class.

.. important::

   If you are calculating a thermal sequence, our recommendation is to calculate the models at low internal temperature with     ``P_surf`` = 1000 bar (default), and the models at high internal temperature with ``P_surf=9.5`` bar. Do not calculate all models at 9.5 bar! At low temperatures, the entropy’s slope becomes flat with time, and makes it difficult to integrate the luminosity equation. Specify the surface pressure for each model as shown in the example below.


Here is an example:

In [ ]:
# Import GASTLI thermal module
import gastli.Thermal_evolution as therm
import gastli.constants as cte
# Other Python modules
import numpy as np
import matplotlib.pyplot as plt
# Create thermal evolution class object
my_therm_obj = therm.thermal_evolution()
# Input for interior
M_P = 100.     # Earth units
# Equilibrium temperatures
Teqpl = 700.
# Core mass fraction
CMF = 0.2
log_FeH = 1.
Tint_array = np.asarray([50., 100., 200., 300., 400., 500., 600., 700., 800.])
# Specify the surface pressure of each model in the thermal sequence
# Models with Tint=50 to 300 K have Psurf=1000 bar, while Tint=400 to 800 K have Psurf=9.5 bar
P_surf_array = np.asarray([1e3, 1e3, 1e3, 1e3, 9.5, 9.5, 9.5, 9.5, 9.5])
my_therm_obj.main(M_P, CMF, Teqpl, Tint_array, log_FeH=log_FeH,P_surf=P_surf_array)
my_therm_obj.solve_thermal_evol_eq(t_Gyr=np.linspace(2.1e-6, 15., 10000))